In [1]:
%%time
#from __future__ import print_function

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pylab
import pandas as pd
import random


moocs = pd.read_csv('~/MOOC_CleanedDataSet.csv',index_col=0)

moocs_sample = moocs.sample(frac=0.67, replace=False) #sample frac%

moocs_label = np.asarray(pd.DataFrame(moocs_sample, columns = ['Grades_Category']))
moocs_label = moocs_label.flatten()

#del moocs['Video']
#del moocs['Forums']
del moocs_sample['Grades_Category']

moocs_train = np.asarray(moocs_sample)

print (str(moocs_train.shape) + " " + str(moocs_label.shape))

moocs_sample1 = moocs.drop(moocs_sample.index) #create the test set

moocs_label1 = np.asarray(pd.DataFrame(moocs_sample1, columns = ['Grades_Category']))
moocs_label1 = moocs_label1.flatten()

#del moocs['Video']
#del moocs['Forums']
del moocs_sample1['Grades_Category']

moocs_test = np.asarray(moocs_sample1)

print (str(moocs_test.shape) + " " + str(moocs_label1.shape))

#moocs_train/moocs_label, moocs_test/moocs_label1
rdd_train = sc.broadcast(moocs_train)
rdd_train_label = sc.broadcast(moocs_label)

rdd_test = sc.broadcast(moocs_test)
rdd_test_label = sc.broadcast(moocs_label1)

#NN model defined here
def map_fun(arr):

    learning_rate_ = arr[0]
    neurons_ = arr[1]
    import tensorflow as tf
    import matplotlib.pyplot as plt
    import numpy as np
    import pylab
    import pandas as pd
    import random

    moocs_train = rdd_train.value
    moocs_label = rdd_train_label.value
    moocs_test = rdd_test.value
    moocs_label1 = rdd_test_label.value

    # Parameters
    learning_rate = learning_rate_
    num_steps = 800 #each step takes in 1 batch_size
    batch_size = 128 #size of each training iteration
    
    print ('number of passes: ' + str((num_steps*batch_size)/moocs_train.shape[0]))
    display_step = 100 #not used

    # Network Parameters
    #the optimal size of the hidden layer is usually between the size of the input and size of the output layers
    n_hidden_1 = neurons_ # 1st layer number of neurons
    n_hidden_2 = neurons_ # 2nd layer number of neurons
    num_input = moocs_train.shape[1] # number of features per input
    num_classes = 2 # predict 0/1 ~bad grade/good grade (>=60%)

    # Define the input function for training
    input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'images': moocs_train}, y=moocs_label,
        batch_size=batch_size, num_epochs=None, shuffle=True)

    # Define the neural network
    def neural_net(x_dict):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict['images']
        # Hidden fully connected layer with 256 neurons
        layer_1 = tf.layers.dense(x, n_hidden_1)
        # Hidden fully connected layer with 256 neurons
        layer_2 = tf.layers.dense(layer_1, n_hidden_2)
        # Output fully connected layer with a neuron for each class
        out_layer = tf.layers.dense(layer_2, num_classes)
        return out_layer

    # Define the model function (following TF Estimator Template)
    def model_fn(features, labels, mode):

        # Build the neural network
        logits = neural_net(features)

        # Predictions
        pred_classes = tf.argmax(logits, axis=1)
        pred_probas = tf.nn.softmax(logits)

        # If prediction mode, early return
        if mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 

        # Define loss and optimizer
        loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
        #tf.nn.softmax_cross_entropy_with_logits for floats
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())

        # Evaluate the accuracy of the model
        acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes) #number of rights/total
        recall = tf.metrics.recall(labels=labels, predictions=pred_classes)
        precision = tf.metrics.precision(labels=labels, predictions=pred_classes)

        # TF Estimators requires to return a EstimatorSpec, that specify
        # the different ops for training, evaluating, ...
        estim_specs = tf.estimator.EstimatorSpec(
          mode=mode,
          predictions=pred_classes,
          loss=loss_op,
          train_op=train_op,
          eval_metric_ops={'accuracy': acc_op, 
                           'recall': recall,
                           'precision': precision
                          })

        return estim_specs

    # Build the Estimator
    model = tf.estimator.Estimator(model_fn)

    # Train the Model
    model.train(input_fn, steps=num_steps)

    # Evaluate the Model
    # Define the input function for evaluating


    input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'images': moocs_test}, y=moocs_label1,
        batch_size=batch_size, shuffle=False)
    # Use the Estimator 'evaluate' method
    results_dic = model.evaluate(input_fn) #
    results_dic.update({'f1': (2*((results_dic['precision']*results_dic['recall'])/(results_dic['precision']+results_dic['recall'])))})
    results_dic.update({'learning_rate':learning_rate_})
    results_dic.update({'neurons':neurons_})
    #print (results_dic)
    #print ('f1 score: ' + str(2*((results_dic['precision']*results_dic['recall'])/(results_dic['precision']+results_dic['recall']))))
    return results_dic
#-------------------------------------------------------------------------------------end of map_fun def    
#hyperparameter tuning
rdd = sc.parallelize([[0.01,40],[0.1,40],[0.3,40],[0.01,31],[0.1,31],[0.3,31]])#[learning rate, neurons per layer]
arr = rdd.map(map_fun).collect() #returns an array of dicts

for i in arr:
    print ('learning rate: ' + str(i['learning_rate']) + ' neurons/layer: ' + str(i['neurons']) + ' accuracy: ' + str(i['accuracy']) + ' f1: ' + str(i['f1']))

(327686, 59) (327686,)
(161397, 59) (161397,)
learning rate: 0.01 neurons/layer: 40 accuracy: 0.965006 f1: 0.00738137075678
learning rate: 0.1 neurons/layer: 40 accuracy: 0.984287 f1: 0.768444120884
learning rate: 0.3 neurons/layer: 40 accuracy: 0.98443 f1: 0.763437747955
learning rate: 0.01 neurons/layer: 31 accuracy: 0.966635 f1: 0.100851558149
learning rate: 0.1 neurons/layer: 31 accuracy: 0.984628 f1: 0.771610081196
learning rate: 0.3 neurons/layer: 31 accuracy: 0.984944 f1: 0.782219111919
CPU times: user 4.31 s, sys: 13.8 s, total: 18.1 s
Wall time: 57.2 s
